In [19]:
# Set these for your specific use case:
semester_start = "08/19/2019"
semester_end = "12/05/2019"
url = 'https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1198/class_list.html?subject=CS'

In [20]:
# Imports and setup 
from bs4 import BeautifulSoup
import urllib.request

from ics import Calendar, Event
from collections import namedtuple
from datetime import datetime, time, date, timedelta
import re

ClassInfo = namedtuple('ClassInfo', 
                       ['number', 
                        'section', 
                        'component', 
                        'title', 
                        'days', 
                        'time', 
                        'location', 
                        'instructor'])

In [21]:
# Helper methods
def generate_class_info(columns):
    time_day = cols[7].text.strip().split(" / ")
    if "TBA" in time_day:
        return None
    
    if '\n' in time_day[1]:
        time = time_day[1].split()[0]
    else:
        time = time_day[1]
        
    if '\n' in cols[10].text.strip():
        split_instructor = cols[10].text.strip().split('\n                                    \n                                    \n                                        ')
        instructor = ";".join(split_instructor)
    else:
        instructor = cols[10].text.strip()
        
    
    return ClassInfo(cols[2].text.strip(),
                    cols[3].text.strip(),
                    cols[4].text.strip(),
                    cols[6].text.strip(),
                    time_day[0],
                    time,
                    cols[8].text.strip(),
                    instructor)

def parse_days(days):
    ret = []
    n = 2
    
    for day in [days[i:i+n] for i in range(0, len(days), n)]:
        if day == "Mo":
            ret.append(1)
        elif day == "Tu":
            ret.append(2)
        elif day == "We":
            ret.append(3)
        elif day == "Th":
            ret.append(4)
        elif day == "Fr":
            ret.append(5)
            
    return ret

def parse_time(time):
    return time.split("-")    
    
def generate_time_delta(time):
    hours = int(time[0:2]) + (12 if time[-2:] == "PM" and time[0:2] != "12" else 0)
    minutes = int(time[3:5])
    return timedelta(hours=hours,minutes=minutes)
    
def generate_event(class_info: ClassInfo, date: datetime):
    start_time = parse_time(class_info.time)[0]
    end_time = parse_time(class_info.time)[1]
    
    stime = date + generate_time_delta(start_time)
    etime = date + generate_time_delta(end_time)
    
    e = Event()
    e.name = f"CS {class_info.number}-{class_info.section} {class_info.component}"
    e.begin = stime.strftime("%Y%m%dT%H:%M:%S")
    e.end = etime.strftime("%Y%m%dT%H:%M:%S")
    e.location = class_info.location
    e.description = f"Taught by {class_info.instructor}"
    
    return e

def generate_calendar(classes, current_day, semester_end_day):
    c = Calendar()
    events = []
    
    # Create events for a single week
    for i in range(7):
        cur_day_of_week = current_day.isoweekday()
        for class_info in classes:
            if class_info is None:
                continue
                
            if cur_day_of_week in parse_days(class_info.days):
                events.append(generate_event(class_info, current_day))
                
        current_day += timedelta(days=1)
        
    c.events = events
    return c

In [22]:
classes = []

with urllib.request.urlopen(url) as response:
    html = response.read().decode('utf-8')

    bs = BeautifulSoup(html, 'html5lib')
    table = bs.find(id='classDetailsTable')
    # print(table)
    
    table_body = table.find('tbody')
    # print(table_body)
    
    rows = table.find_all('tr')
    for row in rows[1:]:
        if "notes" not in row.attrs['class']:
            cols = row.find_all('td')
            classes.append(generate_class_info(cols))
            
# print(classes)

c = Calendar()
current_day = datetime.strptime(semester_start, "%m/%d/%Y")
semester_end_day = datetime.strptime(semester_end, "%m/%d/%Y")
until_date = re.sub("[-]|:","", semester_end_day.isoformat()) + "Z"
    
c = generate_calendar(classes, current_day, semester_end_day)
str_c = str(c).replace("DTSTART", "DTSTART;TZID=America/Denver").replace("DTEND", f"RRULE:FREQ=WEEKLY;UNTIL={until_date}\nDTEND;TZID=America/Denver")
print(str_c)
with open("cs_class_schedule.ics", "w") as file:
    file.writelines(str_c)

BEGIN:VCALENDAR
PRODID:ics.py - http://git.io/lLljaA
VERSION:2.0
BEGIN:VEVENT
DTSTAMP:20190815T053439Z
DTSTART;TZID=America/Denver:20190819T150000Z
RRULE:FREQ=WEEKLY;UNTIL=20191205T000000Z
DTEND;TZID=America/Denver:20190819T162000Z
SUMMARY:CS 534-001 Lecture
DESCRIPTION:Taught by RILOFF\, E. M.
LOCATION:WEB L105
TRANSP:OPAQUE
UID:71c23b50-a813-4e5b-9c88-12b72d7c8c1a@71c2.org
END:VEVENT
BEGIN:VEVENT
DTSTAMP:20190815T053439Z
DTSTART;TZID=America/Denver:20190819T140000Z
RRULE:FREQ=WEEKLY;UNTIL=20191205T000000Z
DTEND;TZID=America/Denver:20190819T145000Z
SUMMARY:CS 1030-001 Lecture
DESCRIPTION:Taught by REGEHR\, J.
LOCATION:MEK 3550
TRANSP:OPAQUE
UID:6d4580e2-97df-462f-9ace-b54babb8e8e4@6d45.org
END:VEVENT
BEGIN:VEVENT
DTSTAMP:20190815T053439Z
DTSTART;TZID=America/Denver:20190819T150000Z
RRULE:FREQ=WEEKLY;UNTIL=20191205T000000Z
DTEND;TZID=America/Denver:20190819T162000Z
SUMMARY:CS 1410-001 Lecture
DESCRIPTION:Taught by ZACHARY\, J. L.
LOCATION:WEB L104
TRANSP:OPAQUE
UID:91101f1a-efe6-42d0-9